In [1]:
#Importing Libraries
import cv2 as cv
import numpy as np

In [2]:
# Loading an image and setting the frame size for the scanned documents
img = cv.imread("docs.jpg")
frameWidth = 480
frameHeight = 640
cv.imshow("Original",img)
cv.waitKey(0)

-1

In [3]:
#Processing the image
def imageProcessing(img):
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    blur = cv.GaussianBlur(gray, (5, 5), 1)
    canny = cv.Canny(blur, 100, 100)
    return canny
canny = imageProcessing(img)
cv.imshow('Canny', canny)
cv.waitKey(0)

-1

In [4]:
#Getting the boundaries of the document
def GetContour(img):
    biggest = np.array([])
    bArea = 0
    contours, heirarchy = cv.findContours(img, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_NONE)
    for cnt in contours:
        area = cv.contourArea(cnt)
        if area > 5500:
            peri = cv.arcLength(cnt, True)
            approx = cv.approxPolyDP(cnt, 0.02*peri, True)
            if area > bArea and len(approx) == 4:
                biggest = approx
                bArea = area
    return biggest
biggest = GetContour(canny)
print(biggest)
print(biggest.shape)

[[[ 53  32]]

 [[ 53 605]]

 [[458 605]]

 [[458  32]]]
(4, 1, 2)


In [5]:
#Getting the points around the boundaries
def reorder(myPoints):
    myPoints = myPoints.reshape((4, 2))
    newPoints = np.zeros((4, 2, ), dtype=np.int32)
    add = myPoints.sum(1)
    newPoints[0] = myPoints[add.argmin()]
    newPoints[3] = myPoints[add.argmax()]
    diff = np.diff(myPoints,axis = 1)
    newPoints[1] = myPoints[np.argmin(diff)]
    newPoints[2] = myPoints[np.argmax(diff)]
    return newPoints
reorder(biggest)

array([[ 53,  32],
       [458,  32],
       [ 53, 605],
       [458, 605]])

In [10]:
#Warping the document
def getWarp(img,biggest):
    biggest = reorder(biggest)
    pts1 = np.float32(biggest)
    pts2 = np.float32([[0, 0], [frameWidth, 0], [0, frameHeight], [frameWidth, frameHeight]])
    matrix = cv.getPerspectiveTransform(pts1, pts2)
    newImg = cv.warpPerspective(img, matrix, (frameWidth, frameHeight))
    return newImg

outputImg = getWarp(img, biggest)
cv.imshow('Scanned', outputImg)
cv.waitKey(0)

-1

In [9]:
img = cv.resize(img,(frameWidth,frameHeight))
result = imageProcessing(img)
biggest = GetContour(result)
imgWrapped = getWarp(img, biggest)
cv.imshow("Output",imgWrapped)
cv.waitKey(0)

-1